# Import packages & Initialize GEE

In [1]:
import ee
import geemap


In [2]:
ee.Authenticate()
ee.Initialize(project='tz-sgr')

# Define spatial unit

All labels and inputs are defined on the Sentinel-2 native 10 m grid.
Each pixel represents a 10 m × 10 m area in UTM coordinates.
All vector data are rasterized to this grid.

In [3]:
# Load Sentiel-2 GeoTIFF image from GEE in 2021
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

s2_2021 = (
    s2
    .filterDate('2021-01-01', '2021-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
)

aoi = ee.Geometry.Rectangle([
    39.0, -6.95,   # min lon, min lat
    39.5, -6.6    # max lon, max lat
])

s2_2021 = s2_2021.filterBounds(aoi)

s2_ref = s2_2021.first().select('B4')
s2_ref_proj = s2_ref.projection()

print(s2_ref_proj.getInfo())
print("Resolution:", s2_ref_proj.nominalScale().getInfo())

# IMPORTANT:
# Earth Engine reducers (e.g. median) drop native projections.
# We explicitly reproject the composite to the Sentinel-2 B4 native grid
# to define the spatial unit (10 m UTM).

s2_2021_img = (
    s2_2021
    .median()
    .select('B4')
    .reproject(s2_ref_proj)
    .clip(aoi)
)

s2_proj = s2_2021_img.projection()

print(s2_proj.getInfo())
print("Resolution:", s2_proj.nominalScale().getInfo())



{'type': 'Projection', 'crs': 'EPSG:32737', 'transform': [10, 0, 499980, 0, -10, 9300040]}
Resolution: 10
{'type': 'Projection', 'crs': 'EPSG:32737', 'transform': [10, 0, 499980, 0, -10, 9300040]}
Resolution: 10


# Get Google 2.5D Building Data as Training Labels

In [4]:
buildings  = ee.ImageCollection("GOOGLE/Research/open-buildings-temporal/v1").filterDate('2021-01-01', '2021-12-31').filterBounds(aoi)
print(buildings.first().getInfo())

{'type': 'Image', 'bands': [{'id': 'building_fractional_count', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1348080, 4149320], 'crs': 'EPSG:32737', 'crs_transform': [0.5, 0, 153600, 0, -0.5, 10009636]}, {'id': 'building_height', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1348080, 4149320], 'crs': 'EPSG:32737', 'crs_transform': [0.5, 0, 153600, 0, -0.5, 10009636]}, {'id': 'building_presence', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1348080, 4149320], 'crs': 'EPSG:32737', 'crs_transform': [0.5, 0, 153600, 0, -0.5, 10009636]}], 'version': 1726589451233073, 'id': 'GOOGLE/Research/open-buildings-temporal/v1/19_EPSG_32737_2021_06_30', 'properties': {'system:time_start': 1625036400000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[41.96518733302105, -7.015203450919197], [41.95287265430302, -4.672336688208628], [41.945536023246746, -2.3292931861383304], [41.94311976984105, 0.08706639585625

In [9]:
# Mosaic all tiles in your AOI into a single image
mosaic = (
    buildings
    .mosaic()
    .setDefaultProjection(s2_proj)
)

# Then select bands
building_density = mosaic.select('building_fractional_count')
building_height = mosaic.select('building_height')
building_presence = mosaic.select('building_presence')

# Clip, resample and reproject 
building_density_resampled = (
    building_density
    .clip(aoi)
    .reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=256
    )
    .reproject(crs=s2_proj, scale=10)
)

building_height_resampled = (
    building_height
    .clip(aoi)
    .reduceResolution(
        reducer=ee.Reducer.mean(),
        maxPixels=256
    )
    .reproject(crs=s2_proj, scale=10)
)

building_presence_resampled = (
    building_presence
    .reduceResolution(
        reducer=ee.Reducer.max(),  # Attention: I am using max here. Also: according to official documentation, thiese confidence values are not calibrated!
        maxPixels=256
    )
    .reproject(crs=s2_proj, scale=10)
)


In [10]:
Map = geemap.Map()
Map.centerObject(aoi, 12)  # zoom into Dar es Salaam

# Sentinel-2 RGB (optional, for reference)
Map.addLayer(s2_2021_img, {'min': 0, 'max': 3000}, 'Sentinel-2 R')

# Building density
Map.addLayer(
    building_density_resampled,
    {'min': 0, 'max': 0.02, 'palette': ['white', 'red']},  # 2% pixel coverage
    'Building Density'
)

# Building height
Map.addLayer(
    building_height_resampled,
    {'min': 0, 'max': 50, 'palette': ['white', 'blue']},  # adjust max as needed
    'Building Height'
)

# Building presence
Map.addLayer(
    building_presence_resampled,
    {'min': 0, 'max': 1, 'palette': ['white', 'black']},
    'Building Presence'
)

Map


Map(center=[-6.7750427264183255, 39.24999999999979], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Stack labels into one image

labels = ee.Image.cat([
    building_density_resampled.rename('density'),
    building_height_resampled.rename('height'),
    building_presence_resampled.rename('presence')
])



# GEE Embeddings

In [ ]:
# Train a classifier

var embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL');

var embeddingsFiltered = embeddings
  .filter(ee.Filter.date(startDate, endDate))
  .filter(ee.Filter.bounds(geometry));

var embeddingsImage = embeddingsFiltered.mosaic();

// Overlay the samples on the image to get training data.
var training = embeddingsImage.sampleRegions({
  collection: gcps,
  properties: ['landcover'],
  scale: 10
});

print('Training Feature', training.first());

var classifier = 
